# TPS 12 - Feature Engineering

In this notebook we test out some feature engineering techniques using LightGBM to see if we get improvements over the baseline.

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 6
TRAIN_SIZE = 500000

In [2]:
import numpy as np
import pandas as pd
import time
import os
import pyarrow
import gc

# Model/Evaluation
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.inspection import permutation_importance
from lightgbm import LGBMClassifier

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load and Prepare Data

In [3]:
# Load full training data
train = pd.read_feather('../data/train.feather')

# Drop low/no variance 
train.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)
train = train[train.Cover_Type != 5]

# Label Encoding
new_encoder = LabelEncoder()
train["Cover_Type"] = new_encoder.fit_transform(train["Cover_Type"])

# Split synthetic data
train, test = train_test_split(
    train, 
    train_size = TRAIN_SIZE, 
    random_state = RANDOM_SEED,
    stratify = train['Cover_Type'],
)
y_train = train['Cover_Type']


# features, data structure for summary scores
features = [x for x in train.columns if x not in ['Id','Cover_Type']]
nonsoil = [x for x in features if not x.startswith('Soil_Type')]
new_rows = list()
gc.collect()

print(f'Training Size: {train.shape[0]} rows, {train.shape[1]} cols')
print(f'Holdout Size: {test.shape[0]} rows, {test.shape[1]} cols\n')

Training Size: 500000 rows, 53 cols
Holdout Size: 3499999 rows, 53 cols



# Model 

In [4]:
# LightGBM Classifier
lgbm_pipeline = make_pipeline(
    LGBMClassifier(
        unbalanced_sets = True,
        metric = 'multi_logloss',
        random_state = RANDOM_SEED,
        n_jobs = 4,
    ),
)

# Scoring Function

In [5]:
def score_features(sklearn_model, processing = None):
    
    # Original Training/Test Split
    features = [x for x in train.columns if x not in ['Id','Cover_Type']]
    X_temp, X_test = train[features], test[features]
    y_temp, y_test = train['Cover_Type'], test['Cover_Type']
    
    # Feature Engineering
    if processing:
        X_temp = processing(X_temp)
        X_test = processing(X_test)
    
    # Store the out-of-fold predictions
    test_preds = np.zeros((X_test.shape[0],6))
    oof_preds = np.zeros((X_temp.shape[0],))
    fi_scores = np.zeros((X_temp.shape[1],))
    scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train[features],train['Cover_Type'])):
       
        # Training and Validation Sets
        X_train, X_valid = X_temp.iloc[train_idx], X_temp.iloc[valid_idx]
        y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]
        
        # Create model
        start = time.time()
        model = clone(sklearn_model)
        model.fit(X_train, y_train)

        # Permutation Importance
        result = permutation_importance(
            model, X_valid, y_valid, 
            random_state=RANDOM_SEED
        )
        fi_scores += result.importances_mean / NUM_FOLDS

        # validation/holdout predictions
        valid_preds = np.ravel(model.predict(X_valid))
        oof_preds[valid_idx] = valid_preds
        test_preds += model.predict_proba(X_test)

        # Save scores and times
        scores[fold] = accuracy_score(y_valid, valid_preds)
        end = time.time()
        times[fold] = end-start
        print(f'Fold {fold} Accuracy:  {round(scores[fold], 5)} in {round(end-start,2)}s.')
        time.sleep(0.5)
    
    features = [x for x in X_temp.columns]
    nonsoil = [x for x in X_test.columns if not x.startswith('Soil_Type')]
    test_preds = np.argmax(test_preds, axis = 1)
    test_score = accuracy_score(y_test, test_preds)
    #print('\n'+model.__class__.__name__)
    print("Train Accuracy:", round(scores.mean(), 5))
    print('Test Accuracy:', round(test_score, 5))
    print(f'Training Time: {round(times.sum(), 2)}s')
    
    fi_scores = pd.Series(
        data = fi_scores, 
        index = features
    ).loc[nonsoil].sort_values()
    
    return scores.mean(), oof_preds, test_score, fi_scores

# Baseline

In [6]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline
)

new_rows.append((
    'Baseline', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94443 in 272.2s.
Fold 1 Accuracy:  0.9362 in 275.13s.
Fold 2 Accuracy:  0.94415 in 258.13s.
Fold 3 Accuracy:  0.944 in 276.86s.
Fold 4 Accuracy:  0.94695 in 287.45s.
Fold 5 Accuracy:  0.94776 in 296.22s.
Train Accuracy: 0.94391
Test Accuracy: 0.95482
Training Time: 1666.0s


Hillshade_9am                        -0.000120
Hillshade_3pm                        -0.000041
Aspect                               -0.000017
Slope                                -0.000016
Wilderness_Area2                      0.000030
Hillshade_Noon                        0.000382
Wilderness_Area4                      0.003027
Horizontal_Distance_To_Hydrology      0.009595
Wilderness_Area1                      0.013958
Vertical_Distance_To_Hydrology        0.016134
Wilderness_Area3                      0.019036
Horizontal_Distance_To_Fire_Points    0.028348
Horizontal_Distance_To_Roadways       0.043016
Elevation                             0.452949
dtype: float64

# Feature Engineering

1. Aspect Features
2. Hillshade Features
3. Water Features
4. Count Features
5. Water/Fire Interactions
6. Roadway Interactions
7. Elevation Interactions

## 1. Aspect Features

In [7]:
def aspect_features(data):
    df = data.copy()
    df['Aspect_360'] = df['Aspect'] % 360
    df['Aspect_Sine'] = (df['Aspect']* np.pi / 180).apply(np.sin)
    df['Aspect_Alt'] = (df['Aspect']-180).where(
        df['Aspect']+180 > 360, df['Aspect'] + 180
    )
    return df

In [8]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    aspect_features
)

new_rows.append((
    'Aspect_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94413 in 260.71s.
Fold 1 Accuracy:  0.9392 in 254.56s.
Fold 2 Accuracy:  0.91787 in 244.72s.
Fold 3 Accuracy:  0.94704 in 297.98s.
Fold 4 Accuracy:  0.9442 in 302.07s.
Fold 5 Accuracy:  0.94329 in 293.29s.
Train Accuracy: 0.93929
Test Accuracy: 0.95278
Training Time: 1653.33s


Hillshade_3pm                         0.000027
Wilderness_Area2                      0.000034
Hillshade_9am                         0.000073
Slope                                 0.000077
Hillshade_Noon                        0.000323
Aspect                                0.001916
Aspect_360                            0.001924
Wilderness_Area4                      0.004480
Aspect_Alt                            0.005071
Aspect_Sine                           0.005268
Horizontal_Distance_To_Hydrology      0.009050
Wilderness_Area1                      0.015148
Vertical_Distance_To_Hydrology        0.015500
Wilderness_Area3                      0.018302
Horizontal_Distance_To_Fire_Points    0.025532
Horizontal_Distance_To_Roadways       0.041620
Elevation                             0.448825
dtype: float64

## 2. Hillshade Features

In [9]:
def hillshade_features(data):
    df = data.copy()
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

    # Clip Range
    df["Hillshade_9am_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    df["Hillshade_Noon_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    df["Hillshade_3pm_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    
    # Hillshade
    #df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    df["Hillshade_Sum"] = df[shade_features].sum(axis=1)
    df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    
    return df

In [10]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    hillshade_features
)

new_rows.append((
    'Hillshade_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.92304 in 283.48s.
Fold 1 Accuracy:  0.93136 in 264.04s.
Fold 2 Accuracy:  0.94757 in 320.1s.
Fold 3 Accuracy:  0.93933 in 298.99s.
Fold 4 Accuracy:  0.94712 in 314.17s.
Fold 5 Accuracy:  0.94184 in 292.95s.
Train Accuracy: 0.93838
Test Accuracy: 0.95227
Training Time: 1773.73s


Hillshade_3pm_Clipped                 0.000000
Hillshade_Noon_Clipped                0.000000
Aspect                                0.000014
Wilderness_Area2                      0.000039
Slope                                 0.000051
Hillshade_Noon                        0.000534
Hillshade_9am_Clipped                 0.000787
Hillshade_Range                       0.001398
Hillshade_9am                         0.001747
Hillshade_Sum                         0.002125
Hillshade_3pm                         0.002275
Wilderness_Area4                      0.002320
Horizontal_Distance_To_Hydrology      0.008329
Wilderness_Area1                      0.013633
Vertical_Distance_To_Hydrology        0.015226
Wilderness_Area3                      0.017461
Horizontal_Distance_To_Fire_Points    0.026985
Horizontal_Distance_To_Roadways       0.040217
Elevation                             0.448541
dtype: float64

## 3. Water Features

In [11]:
# Helper function
def start_at_eps(series, eps=1e-10): 
    return series - series.min() + eps

def water_features(data):
    df = data.copy()
    
    # use float64 for squaring
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float64')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float64')
    pos_h_hydrology = start_at_eps(df["Horizontal_Distance_To_Hydrology"])
    pos_v_hydrology = start_at_eps(df['Vertical_Distance_To_Hydrology'])
    
    # Manhatten Distances
    df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    df['Hydro_Taxicab_Pos'] = (pos_h_hydrology ** 2 + pos_v_hydrology ** 2).apply(np.sqrt).rename('Euclidean_positive_hydrology').astype(np.float32)
    
    # Euclidean Distance
    df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    df['Hydro_Euclid_Pos'] = (pos_h_hydrology ** 2 + pos_v_hydrology ** 2).apply(np.sqrt)
    
    # Misc Features
    df['Water_Direction'] = df['Vertical_Distance_To_Hydrology'].apply(np.sign)
    df['Water Elevation'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    
    # Store each as float32
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float32')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float32')
    df["Hydro_Taxicab"] = df["Hydro_Taxicab"].astype('float32')
    df['Hydro_Taxicab_Pos'] = df['Hydro_Taxicab_Pos'].astype('float32')
    df["Hydro_Euclid"] = df["Hydro_Euclid"].astype('float32')
    df['Hydro_Euclid_Pos'] = df['Hydro_Euclid_Pos'].astype('float32')
    df['Water_Direction'] = df['Water_Direction'].astype('float32')
    df['Water Elevation'] = df['Water Elevation'].astype('float32')
    
    return df

In [13]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    water_features
)

new_rows.append((
    'Water_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94683 in 304.74s.
Fold 1 Accuracy:  0.94283 in 299.81s.
Fold 2 Accuracy:  0.93885 in 281.48s.
Fold 3 Accuracy:  0.94562 in 318.33s.
Fold 4 Accuracy:  0.94908 in 309.24s.
Fold 5 Accuracy:  0.95115 in 321.63s.
Train Accuracy: 0.94573
Test Accuracy: 0.95612
Training Time: 1835.22s


Hillshade_9am                        -0.000026
Hydro_Euclid_Pos                      0.000000
Wilderness_Area2                      0.000038
Slope                                 0.000064
Water_Direction                       0.000084
Aspect                                0.000266
Hillshade_3pm                         0.000274
Hillshade_Noon                        0.000351
Wilderness_Area4                      0.002214
Vertical_Distance_To_Hydrology        0.002497
Hydro_Euclid                          0.005372
Hydro_Taxicab_Pos                     0.008280
Horizontal_Distance_To_Hydrology      0.009444
Hydro_Taxicab                         0.010491
Wilderness_Area1                      0.016258
Wilderness_Area3                      0.019786
Horizontal_Distance_To_Fire_Points    0.027995
Horizontal_Distance_To_Roadways       0.043686
Water Elevation                       0.158682
Elevation                             0.307295
dtype: float64

## 4. Count Features

In [14]:
def count_features(data):
    
    df = data.copy()
    soil_features = [x for x in df.columns if x.startswith("Soil_Type")]
    wilderness_features = [x for x in df.columns if x.startswith("Wilderness_Area")]

    # Count features
    df["Soil_Count"] = df[soil_features].apply(sum, axis=1)
    df["Wilderness_Count"] = df[wilderness_features].apply(sum, axis=1)
    
    return df

In [15]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    count_features
)

new_rows.append((
    'Count_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95079 in 278.72s.
Fold 1 Accuracy:  0.9143 in 252.55s.
Fold 2 Accuracy:  0.93394 in 246.34s.
Fold 3 Accuracy:  0.94997 in 293.68s.
Fold 4 Accuracy:  0.94689 in 294.6s.
Fold 5 Accuracy:  0.94736 in 292.33s.
Train Accuracy: 0.94054
Test Accuracy: 0.95564
Training Time: 1658.22s


Hillshade_3pm                         0.000057
Slope                                 0.000058
Aspect                                0.000099
Hillshade_9am                         0.000124
Hillshade_Noon                        0.000326
Wilderness_Area2                      0.000873
Wilderness_Count                      0.003499
Wilderness_Area4                      0.004563
Horizontal_Distance_To_Hydrology      0.008950
Vertical_Distance_To_Hydrology        0.015376
Wilderness_Area1                      0.021082
Wilderness_Area3                      0.025540
Horizontal_Distance_To_Fire_Points    0.026913
Soil_Count                            0.033297
Horizontal_Distance_To_Roadways       0.042271
Elevation                             0.449533
dtype: float64

## 5. Water/Fire Interactions

In [16]:
def hydrofire_interactions(data):
    
    df = data.copy()
    df['Hydro_Fire_Sum'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_AbsDiff'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Fire_EpsSum'] = start_at_eps(df['Horizontal_Distance_To_Hydrology']) + start_at_eps(df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Fire_Diff'] = df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points']
    return df

In [17]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    hydrofire_interactions
)

new_rows.append((
    'Water_Fire', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94677 in 301.21s.
Fold 1 Accuracy:  0.92091 in 269.9s.
Fold 2 Accuracy:  0.94805 in 319.68s.
Fold 3 Accuracy:  0.94949 in 308.66s.
Fold 4 Accuracy:  0.94896 in 310.22s.
Fold 5 Accuracy:  0.94946 in 312.4s.
Train Accuracy: 0.94394
Test Accuracy: 0.95501
Training Time: 1822.07s


Hillshade_9am                        -0.000314
Hillshade_3pm                        -0.000130
Aspect                               -0.000094
Slope                                -0.000044
Wilderness_Area2                      0.000030
Hillshade_Noon                        0.000260
Wilderness_Area4                      0.004556
Hydro_Fire_EpsSum                     0.006406
Horizontal_Distance_To_Hydrology      0.006746
Hydro_Fire_Sum                        0.007772
Hydro_Fire_AbsDiff                    0.008016
Horizontal_Distance_To_Fire_Points    0.015686
Vertical_Distance_To_Hydrology        0.015995
Wilderness_Area1                      0.016936
Hydro_Fire_Diff                       0.019240
Wilderness_Area3                      0.022271
Horizontal_Distance_To_Roadways       0.042738
Elevation                             0.452208
dtype: float64

## 6. Roadway Interactions

In [18]:
def roadway_interactions(data):
    df = data.copy()
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    return df

In [19]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    roadway_interactions
)

new_rows.append((
    'Road_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.94708 in 277.89s.
Fold 1 Accuracy:  0.91565 in 272.66s.
Fold 2 Accuracy:  0.94954 in 276.9s.
Fold 3 Accuracy:  0.94581 in 304.64s.
Fold 4 Accuracy:  0.94835 in 315.12s.
Fold 5 Accuracy:  0.94714 in 300.74s.
Train Accuracy: 0.94226
Test Accuracy: 0.95453
Training Time: 1747.95s


Hillshade_3pm                         0.000029
Wilderness_Area2                      0.000033
Aspect                                0.000034
Slope                                 0.000039
Hillshade_9am                         0.000106
Hillshade_Noon                        0.000422
Hydro_Road_1                          0.004452
Wilderness_Area4                      0.005692
Horizontal_Distance_To_Hydrology      0.007011
Hydro_Road_2                          0.008542
Fire_Road_2                           0.010491
Fire_Road_1                           0.013096
Horizontal_Distance_To_Fire_Points    0.013123
Vertical_Distance_To_Hydrology        0.015601
Wilderness_Area1                      0.017474
Wilderness_Area3                      0.021466
Horizontal_Distance_To_Roadways       0.026384
Elevation                             0.450425
dtype: float64

## 7. Elevation Interactions

In [20]:
def elevation_interactions(data):
    df = data.copy()
    df['Road_Elev_Int'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['VHydro_Elev_Int'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['Elev_VHydro_Diff'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['Elev_HHydro_Diff'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    
    return df

In [21]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    lgbm_pipeline, 
    elevation_interactions
)

new_rows.append((
    'Elev_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.9376 in 272.41s.
Fold 1 Accuracy:  0.94898 in 288.11s.
Fold 2 Accuracy:  0.94985 in 303.91s.
Fold 3 Accuracy:  0.95069 in 310.94s.
Fold 4 Accuracy:  0.94404 in 272.29s.
Fold 5 Accuracy:  0.94794 in 305.2s.
Train Accuracy: 0.94652
Test Accuracy: 0.95581
Training Time: 1752.85s


Hillshade_3pm                        -0.000040
Slope                                -0.000036
Hillshade_9am                        -0.000008
Road_Elev_Int                         0.000006
VHydro_Elev_Int                       0.000042
Aspect                                0.000044
Wilderness_Area2                      0.000053
Hillshade_Noon                        0.000499
Vertical_Distance_To_Hydrology        0.002051
Horizontal_Distance_To_Hydrology      0.004451
Wilderness_Area4                      0.005108
Wilderness_Area1                      0.015130
Wilderness_Area3                      0.019100
Horizontal_Distance_To_Fire_Points    0.028362
Horizontal_Distance_To_Roadways       0.045300
Elev_HHydro_Diff                      0.081081
Elev_VHydro_Diff                      0.187686
Elevation                             0.193634
dtype: float64

# Summary

In [22]:
pd.DataFrame.from_records(
    data = new_rows,
    columns = ['features','cv_scores','holdout','recall_0', 'recall_1','recall_2','recall_3','recall_4','recall_5']
).sort_values('holdout')

,features,cv_scores,holdout,recall_0,recall_1,recall_2,recall_3,recall_4,recall_5
2,Hillshade_Features,0.938376,0.952266,0.950435,0.954255,0.802077,0.000000,0.388655,0.612103
1,Aspect_Features,0.939288,0.952776,0.947836,0.956999,0.825090,0.000000,0.402661,0.557369
6,Road_Interactions,0.942262,0.954526,0.950413,0.958739,0.827502,0.000000,0.427871,0.612232
0,Baseline,0.943914,0.954825,0.951084,0.960790,0.833633,0.000000,0.402661,0.613388
5,Water_Fire,0.943940,0.955006,0.953274,0.960348,0.824313,0.021277,0.398459,0.609405
4,Count_Features,0.940540,0.955640,0.945700,0.955698,0.834328,0.000000,0.404062,0.706154
7,Elev_Interactions,0.946518,0.955813,0.954081,0.963230,0.833470,0.021277,0.431373,0.616472
3,Water_Features,0.945726,0.956117,0.954527,0.962247,0.823905,0.042553,0.419468,0.622896
